In [3]:
import cv2
import numpy as np
import pandas as pd
import os
from ultralytics import YOLO

# טוען את מודל YOLO Pose
yolo_model = YOLO('yolo11n-pose.pt')

# גודל חלון זמן בפריימים
WINDOW_SIZE = 60

def calculate_angle(a, b, c):
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

def normalize_data(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

# פונקציה לחישוב ציון תנועה על פי הנוסחה שניתנה
def calculate_movement_score(velocity_avg, angle_score, confidence_score):
    return 0.4 * velocity_avg + 0.3 * angle_score + 0.3 * confidence_score

# מיפוי נקודות מפתח לבחירה
selected_keypoints = {
    "left_shoulder": 5,
    "right_shoulder": 6,
    "left_elbow": 7,
    "right_elbow": 8,
    "left_hip": 11,
    "right_hip": 12,
    "left_knee": 13,
    "right_knee": 14
}

# לזוויות בלבד: כתף-מרפק-שורש כף יד, ירך-ברך-קרסול
selected_angles = {
    'left_elbow': [5, 7, 9],
    'right_elbow': [6, 8, 10],
    'left_knee': [11, 13, 15],
    'right_knee': [12, 14, 16]
}

video_folder = "final_videos"
dataset = []

if not os.path.exists(video_folder):
    print(f"תיקיית הסרטונים '{video_folder}' לא קיימת.")
    exit(1)

video_files = sorted([f for f in os.listdir(video_folder) if f.endswith(".mp4")])

MIN_VELOCITY = -10
MAX_VELOCITY = 10
MIN_ACCELERATION = -10
MAX_ACCELERATION = 10

# יצירת מבנה נתונים ריק לאחסון המידע בחלונות זמן
def create_empty_data_structure():
    return {
        # נתוני כתף שמאל
        "left_shoulder_velocities": [],
        "left_shoulder_confidences": [],
        
        # נתוני כתף ימין
        "right_shoulder_velocities": [],
        "right_shoulder_confidences": [],
        
        # נתוני מרפק שמאל
        "left_elbow_velocities": [],
        "left_elbow_angles": [],
        "left_elbow_confidences": [],
        
        # נתוני מרפק ימין
        "right_elbow_velocities": [],
        "right_elbow_angles": [],
        "right_elbow_confidences": [],
        
        # נתוני ירך שמאל
        "left_hip_velocities": [],
        "left_hip_confidences": [],
        
        # נתוני ירך ימין
        "right_hip_velocities": [],
        "right_hip_confidences": [],
        
        # נתוני ברך שמאל
        "left_knee_velocities": [],
        "left_knee_angles": [],
        "left_knee_confidences": [],
        
        # נתוני ברך ימין
        "right_knee_velocities": [],
        "right_knee_angles": [],
        "right_knee_confidences": []
    }

# פונקציה לחישוב הציונים עבור חלון זמן
def calculate_window_scores(data):
    # חישוב ממוצעים וציונים עבור כל חלקי גוף
    
    # כתף שמאל - רק מהירות וביטחון
    left_shoulder_velocity_avg = np.mean(data["left_shoulder_velocities"]) if data["left_shoulder_velocities"] else 0
    left_shoulder_confidence_avg = np.mean(data["left_shoulder_confidences"]) if data["left_shoulder_confidences"] else 0
    
    # כתף ימין - רק מהירות וביטחון
    right_shoulder_velocity_avg = np.mean(data["right_shoulder_velocities"]) if data["right_shoulder_velocities"] else 0
    right_shoulder_confidence_avg = np.mean(data["right_shoulder_confidences"]) if data["right_shoulder_confidences"] else 0
    
    # מרפק שמאל - מהירות, זווית וביטחון
    left_elbow_velocity_avg = np.mean(data["left_elbow_velocities"]) if data["left_elbow_velocities"] else 0
    left_elbow_angle_avg = np.mean(data["left_elbow_angles"]) if data["left_elbow_angles"] else 0
    left_elbow_confidence_avg = np.mean(data["left_elbow_confidences"]) if data["left_elbow_confidences"] else 0
    
    # מרפק ימין - מהירות, זווית וביטחון
    right_elbow_velocity_avg = np.mean(data["right_elbow_velocities"]) if data["right_elbow_velocities"] else 0
    right_elbow_angle_avg = np.mean(data["right_elbow_angles"]) if data["right_elbow_angles"] else 0
    right_elbow_confidence_avg = np.mean(data["right_elbow_confidences"]) if data["right_elbow_confidences"] else 0
    
    # ירך שמאל - רק מהירות וביטחון
    left_hip_velocity_avg = np.mean(data["left_hip_velocities"]) if data["left_hip_velocities"] else 0
    left_hip_confidence_avg = np.mean(data["left_hip_confidences"]) if data["left_hip_confidences"] else 0
    
    # ירך ימין - רק מהירות וביטחון
    right_hip_velocity_avg = np.mean(data["right_hip_velocities"]) if data["right_hip_velocities"] else 0
    right_hip_confidence_avg = np.mean(data["right_hip_confidences"]) if data["right_hip_confidences"] else 0
    
    # ברך שמאל - מהירות, זווית וביטחון
    left_knee_velocity_avg = np.mean(data["left_knee_velocities"]) if data["left_knee_velocities"] else 0
    left_knee_angle_avg = np.mean(data["left_knee_angles"]) if data["left_knee_angles"] else 0
    left_knee_confidence_avg = np.mean(data["left_knee_confidences"]) if data["left_knee_confidences"] else 0
    
    # ברך ימין - מהירות, זווית וביטחון
    right_knee_velocity_avg = np.mean(data["right_knee_velocities"]) if data["right_knee_velocities"] else 0
    right_knee_angle_avg = np.mean(data["right_knee_angles"]) if data["right_knee_angles"] else 0
    right_knee_confidence_avg = np.mean(data["right_knee_confidences"]) if data["right_knee_confidences"] else 0
    
    # חישוב ציוני תנועה עבור ברכיים ומרפקים (לקביעת דומיננטיות)
    left_elbow_score = calculate_movement_score(left_elbow_velocity_avg, left_elbow_angle_avg, left_elbow_confidence_avg)
    right_elbow_score = calculate_movement_score(right_elbow_velocity_avg, right_elbow_angle_avg, right_elbow_confidence_avg)
    left_knee_score = calculate_movement_score(left_knee_velocity_avg, left_knee_angle_avg, left_knee_confidence_avg)
    right_knee_score = calculate_movement_score(right_knee_velocity_avg, right_knee_angle_avg, right_knee_confidence_avg)
    
    # עבור איברים ללא זווית (כתפיים וירכיים), נשתמש בערך זווית 0 לצורך הנוסחה
    left_shoulder_score = calculate_movement_score(left_shoulder_velocity_avg, 0, left_shoulder_confidence_avg)
    right_shoulder_score = calculate_movement_score(right_shoulder_velocity_avg, 0, right_shoulder_confidence_avg)
    left_hip_score = calculate_movement_score(left_hip_velocity_avg, 0, left_hip_confidence_avg)
    right_hip_score = calculate_movement_score(right_hip_velocity_avg, 0, right_hip_confidence_avg)
    
    # חישוב ציון כללי כממוצע של כל הציונים מכל האיברים
    overall_score = (
        left_elbow_score + right_elbow_score + 
        left_knee_score + right_knee_score + 
        left_shoulder_score + right_shoulder_score + 
        left_hip_score + right_hip_score
    ) / 8
    
    # קביעת איבר דומיננטי
    dominant_knee = "left" if left_knee_score > right_knee_score else "right"
    dominant_elbow = "left" if left_elbow_score > right_elbow_score else "right"
    
    return {
        "overall_movement_score": overall_score,
        "left_knee_score": left_knee_score,
        "right_knee_score": right_knee_score,
        "dominant_knee": dominant_knee,
        "left_elbow_score": left_elbow_score,
        "right_elbow_score": right_elbow_score,
        "dominant_elbow": dominant_elbow,
        "left_shoulder_score": left_shoulder_score,
        "right_shoulder_score": right_shoulder_score,
        "left_hip_score": left_hip_score,
        "right_hip_score": right_hip_score
    }

for video_index, video_file in enumerate(video_files):
    video_id = video_index + 1
    print(f"🔍 מעבד: {video_file} (ID: {video_id})")
    
    cap = cv2.VideoCapture(os.path.join(video_folder, video_file))
    
    if not cap.isOpened():
        print(f"שגיאה בפתיחת הקובץ: {video_file}")
        continue
    
    frame_index = 0
    prev_points = {}
    prev_time = None
    current_window = 0
    
    # מילון לאחסון נתונים לכל חלון זמן
    window_data = {}
    window_data[current_window] = create_empty_data_structure()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # חישוב מספר החלון הנוכחי
        current_window = frame_index // WINDOW_SIZE
        
        # יצירת מבנה נתונים לחלון הנוכחי אם לא קיים
        if current_window not in window_data:
            window_data[current_window] = create_empty_data_structure()
            
        height, width, _ = frame.shape
        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        
        results = yolo_model(frame)
        
        if len(results[0].keypoints) == 0:
            frame_index += 1
            continue
            
        keypoints = results[0].keypoints.data[0].cpu().numpy()
        keypoints[:, 0] *= width
        keypoints[:, 1] *= height
        
        data_entry = {
            "video_id": video_id,
            "frame": frame_index,
            "window_id": current_window
        }
        
        # חישוב זוויות
        angles = {}
        for name, (a_idx, b_idx, c_idx) in selected_angles.items():
            if a_idx < len(keypoints) and b_idx < len(keypoints) and c_idx < len(keypoints):
                a = keypoints[a_idx][:2]
                b = keypoints[b_idx][:2]
                c = keypoints[c_idx][:2]
                angle = calculate_angle(a, b, c)
                angles[name] = angle
                data_entry[f"angle_{name}"] = angle
        
        # איסוף נתונים עבור כל נקודת מפתח
        for name, idx in selected_keypoints.items():
            if idx >= len(keypoints):
                continue
                
            x, y, conf = keypoints[idx]
            x_norm, y_norm = x / width, y / height
            
            data_entry[f"{name}_x"] = x_norm
            data_entry[f"{name}_y"] = y_norm
            data_entry[f"{name}_conf"] = conf
            
            # שמירת הוודאות עבור חישוב ציון התנועה בחלון הנוכחי
            if name == "left_shoulder":
                window_data[current_window]["left_shoulder_confidences"].append(conf)
            elif name == "right_shoulder":
                window_data[current_window]["right_shoulder_confidences"].append(conf)
            elif name == "left_hip":
                window_data[current_window]["left_hip_confidences"].append(conf)
            elif name == "right_hip":
                window_data[current_window]["right_hip_confidences"].append(conf)
            elif name == "left_elbow":
                window_data[current_window]["left_elbow_confidences"].append(conf)
                if "left_elbow" in angles:
                    window_data[current_window]["left_elbow_angles"].append(angles["left_elbow"])
            elif name == "right_elbow":
                window_data[current_window]["right_elbow_confidences"].append(conf)
                if "right_elbow" in angles:
                    window_data[current_window]["right_elbow_angles"].append(angles["right_elbow"])
            elif name == "left_knee":
                window_data[current_window]["left_knee_confidences"].append(conf)
                if "left_knee" in angles:
                    window_data[current_window]["left_knee_angles"].append(angles["left_knee"])
            elif name == "right_knee":
                window_data[current_window]["right_knee_confidences"].append(conf)
                if "right_knee" in angles:
                    window_data[current_window]["right_knee_angles"].append(angles["right_knee"])
            
            # חישוב מהירות אם יש מידע קודם
            if name in prev_points and prev_time is not None:
                time_diff = current_time - prev_time
                if time_diff > 0:
                    prev_x, prev_y = prev_points[name]
                    vx = (x_norm - prev_x) / time_diff
                    vy = (y_norm - prev_y) / time_diff
                    vel_magnitude = np.sqrt(vx**2 + vy**2)
                    
                    # נרמול ערכי המהירות
                    norm_velocity_x = normalize_data(vx, MIN_VELOCITY, MAX_VELOCITY)
                    norm_velocity_y = normalize_data(vy, MIN_VELOCITY, MAX_VELOCITY)
                    
                    data_entry[f"{name}_velocity_x"] = norm_velocity_x
                    data_entry[f"{name}_velocity_y"] = norm_velocity_y
                    
                    # שמירת המהירות לחישוב ציון התנועה בחלון הנוכחי
                    if name == "left_shoulder":
                        window_data[current_window]["left_shoulder_velocities"].append(vel_magnitude)
                    elif name == "right_shoulder":
                        window_data[current_window]["right_shoulder_velocities"].append(vel_magnitude)
                    elif name == "left_elbow":
                        window_data[current_window]["left_elbow_velocities"].append(vel_magnitude)
                    elif name == "right_elbow":
                        window_data[current_window]["right_elbow_velocities"].append(vel_magnitude)
                    elif name == "left_hip":
                        window_data[current_window]["left_hip_velocities"].append(vel_magnitude)
                    elif name == "right_hip":
                        window_data[current_window]["right_hip_velocities"].append(vel_magnitude)
                    elif name == "left_knee":
                        window_data[current_window]["left_knee_velocities"].append(vel_magnitude)
                    elif name == "right_knee":
                        window_data[current_window]["right_knee_velocities"].append(vel_magnitude)
            
            # שמירת המיקום הנוכחי לחישוב בפריים הבא
            prev_points[name] = (x_norm, y_norm)
        
        dataset.append(data_entry)
        prev_time = current_time
        frame_index += 1
    
    cap.release()
    
    # חישוב ציוני תנועה לכל חלון זמן בוידאו הנוכחי
    for window_id, data in window_data.items():
        if not data["left_shoulder_velocities"] and not data["right_shoulder_velocities"]:
            # דילוג על חלונות ריקים
            continue
            
        # חישוב הציונים לחלון הנוכחי
        window_scores = calculate_window_scores(data)
        
        # הוספת מידע על הוידאו והחלון לציונים
        window_scores["video_id"] = video_id
        window_scores["window_id"] = window_id
        
        # עדכון הציונים בשורות המתאימות בדאטאסט
        for entry in dataset:
            if entry["video_id"] == video_id and entry["window_id"] == window_id:
                entry.update(window_scores)

# יצירת הדאטאפריים וכתיבה לקובץ CSV - עם קידוד מפורש לתמיכה בעברית
df = pd.DataFrame(dataset)
df.to_csv("yolo_motion_dataset_with_window_scores.csv", index=False, encoding='utf-8-sig')
print("✅ סיום עיבוד הסרטונים, הנתונים נשמרו בקובץ yolo_motion_dataset_with_window_scores.csv")

🔍 מעבד: video_1.mp4 (ID: 1)

0: 640x384 1 person, 196.7ms
Speed: 29.8ms preprocess, 196.7ms inference, 7.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 150.4ms
Speed: 2.5ms preprocess, 150.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 135.2ms
Speed: 2.6ms preprocess, 135.2ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 154.3ms
Speed: 2.5ms preprocess, 154.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 141.6ms
Speed: 3.0ms preprocess, 141.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 154.8ms
Speed: 2.6ms preprocess, 154.8ms inference, 8.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 146.1ms
Speed: 2.5ms preprocess, 146.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 141.5ms
Speed: 2.9ms preprocess, 141.5ms inference, 1

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



0: 640x384 1 person, 183.4ms
Speed: 5.4ms preprocess, 183.4ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 146.9ms
Speed: 3.3ms preprocess, 146.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 152.5ms
Speed: 4.1ms preprocess, 152.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 168.2ms
Speed: 4.1ms preprocess, 168.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 161.7ms
Speed: 3.9ms preprocess, 161.7ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 211.6ms
Speed: 3.7ms preprocess, 211.6ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 188.4ms
Speed: 4.4ms preprocess, 188.4ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 194.7ms
Speed: 4.6ms preprocess, 194.7ms inference, 2.3ms postprocess per image at

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



0: 640x384 1 person, 188.1ms
Speed: 54.4ms preprocess, 188.1ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 194.5ms
Speed: 3.0ms preprocess, 194.5ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 173.2ms
Speed: 2.6ms preprocess, 173.2ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 163.3ms
Speed: 3.0ms preprocess, 163.3ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 213.0ms
Speed: 9.0ms preprocess, 213.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 160.1ms
Speed: 3.3ms preprocess, 160.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 236.6ms
Speed: 2.6ms preprocess, 236.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 157.7ms
Speed: 2.5ms preprocess, 157.7ms inference, 2.5ms postprocess per image a

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Speed: 2.2ms preprocess, 95.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 99.3ms
Speed: 2.5ms preprocess, 99.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 103.0ms
Speed: 2.4ms preprocess, 103.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 96.1ms
Speed: 2.4ms preprocess, 96.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 117.6ms
Speed: 2.4ms preprocess, 117.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 103.0ms
Speed: 2.5ms preprocess, 103.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 97.1ms
Speed: 2.4ms preprocess, 97.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 94.4ms
Speed: 2.3ms preprocess, 94.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Speed: 3.3ms preprocess, 118.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 117.0ms
Speed: 3.8ms preprocess, 117.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 121.0ms
Speed: 2.5ms preprocess, 121.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 138.6ms
Speed: 4.1ms preprocess, 138.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 132.4ms
Speed: 8.3ms preprocess, 132.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 121.8ms
Speed: 3.0ms preprocess, 121.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 120.5ms
Speed: 3.0ms preprocess, 120.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 114.3ms
Speed: 3.1ms preprocess, 114.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 38

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

